In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm

# # 원본 CSV 파일들이 있는 소스 디렉토리
# source_dir = '법령별 csv 저장 중복제거'

# # 생성된 .txt 파일들을 저장할 출력 디렉토리
# output_dir = '법령_통합텍스트'

# # 합칠 대상 컬럼 목록
# # ★ 만약 다른 컬럼도 추가하고 싶다면 이 리스트에 추가하세요.
# columns_to_combine = ['조문내용', '항', '호', '목']

# # 출력 디렉토리 생성 (이미 존재하면 넘어감)
# os.makedirs(output_dir, exist_ok=True)

# print(f"'{source_dir}' 폴더의 CSV 파일들을 읽어 '{output_dir}' 폴더에 TXT 파일로 통합합니다.")
# print(f"통합 대상 컬럼: {columns_to_combine}")

# # 소스 디렉토리의 모든 파일에 대해 반복 작업
# if not os.path.isdir(source_dir):
#     print(f"오류: 원본 폴더 '{source_dir}'를 찾을 수 없습니다. 경로를 확인해주세요.")
# else:
#     for filename in tqdm(os.listdir(source_dir)):
#         if filename.endswith('.csv'):
#             try:
#                 source_path = os.path.join(source_dir, filename)
#                 df = pd.read_csv(source_path, sep='\t')

#                 # 실제 파일에 존재하는 컬럼만 필터링
#                 existing_columns = [col for col in columns_to_combine if col in df.columns]

#                 # 해당 컬럼들의 빈 값(NaN)을 빈 문자열('')로 대체
#                 df[existing_columns] = df[existing_columns].fillna('').astype(str)

#                 # 각 행(row)별로 존재하는 컬럼 내용을 공백으로 이어붙임
#                 combined_series = df[existing_columns].apply(lambda row: ' '.join(row.values), axis=1)

#                 # 법령 전체의 텍스트를 하나의 문자열로 합침 (각 행은 줄바꿈으로 구분)
#                 full_text = '\n'.join(combined_series)

#                 # 출력 파일 경로 설정 (예: '가사소송법.csv' -> '가사소송법.txt')
#                 output_filename = filename.replace('.csv', '.txt')
#                 output_path = os.path.join(output_dir, output_filename)

#                 # 통합된 텍스트를 .txt 파일로 저장
#                 with open(output_path, 'w', encoding='utf-8') as f:
#                     f.write(full_text)

#             except Exception as e:
#                 print(f"오류 발생: {filename} 처리 중 문제 발생 - {e}")

#     print(f"\n작업 완료! 모든 법령이 '{output_dir}' 폴더에 .txt 파일로 저장되었습니다.")

'법령별 csv 저장 중복제거' 폴더의 CSV 파일들을 읽어 '법령_통합텍스트' 폴더에 TXT 파일로 통합합니다.
통합 대상 컬럼: ['조문내용', '항', '호', '목']


100%|██████████| 1500/1500 [00:20<00:00, 71.74it/s] 


작업 완료! 모든 법령이 '법령_통합텍스트' 폴더에 .txt 파일로 저장되었습니다.


In [ ]:
# # ==============================================================================
# # 1. 라이브러리 및 환경 설정
# # ==============================================================================
# import os
# import pandas as pd
# import numpy as np
# from tqdm import tqdm  # 진행 상황을 시각적으로 보여주는 라이브러리

# # --- 텍스트 처리를 위한 라이브러리 ---
# from konlpy.tag import Okt # Okt이 설치되지 않았다면 Okt로 변경: from konlpy.tag import Okt
# from sklearn.feature_extraction.text import TfidfVectorizer

# # --- 딥러닝 및 유사도 계산을 위한 라이브러리 ---
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# import torch

# # tqdm을 pandas의 apply와 함께 사용하기 위해 필요한 설정
# tqdm.pandas()

# # Okt 초기화 (Okt 설치 경로에 따라 필요시 인자 추가)
# # okt를 사용할 경우: tokenizer = Okt()
# try:
#     tokenizer = Okt()
#     print("Okt 토크나이저를 사용합니다.")
# except Exception as e:
#     print(f"Okt 로드 실패: {e}")
#     print("Okt 토크나이저로 대체합니다. 처리 속도가 느릴 수 있습니다.")
#     from konlpy.tag import Okt
#     tokenizer = Okt()


c:\Users\Administrator\anaconda3\envs\bylaw\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Okt 토크나이저를 사용합니다.


In [ ]:
# import os
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
# from konlpy.tag import Mecab
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# # (이하 필요한 라이브러리들)

# try:
#     # ★ 탭으로 분리된 과목 목록 파일을 읽도록 sep='\t' 추가
#     public_laws = pd.read_csv('변호사시험_출제대상_부속법령(공법).csv')['법령명'].tolist()
#     civil_laws = pd.read_csv('변호사시험_출제대상_부속법령(민사법).csv')['법령명'].tolist()
#     criminal_laws = pd.read_csv('변호사시험_출제대상_부속법령(형사법).csv')['법령명'].tolist()
#     print("과목별 법령 목록 로드 완료.")
# except FileNotFoundError as e:
#     print(f"오류: 과목별 법령 목록 파일을 찾을 수 없습니다. ({e.filename})")
#     exit() # 스크립트에서는 주석 해제

# # --- 전체 법령 데이터 로드 (생성된 .txt 파일 로드) ---
# all_laws_dir = '법령_통합텍스트' # ★ .txt 파일들이 저장된 폴더
# all_laws_data = []


# print(f"'{all_laws_dir}' 폴더에서 전체 법령 텍스트를 로드합니다...")
# if not os.path.isdir(all_laws_dir):
#     print(f"오류: '{all_laws_dir}' 폴더를 찾을 수 없습니다. 1단계 코드를 먼저 실행했는지 확인해주세요.")
#     exit()

# for file_name in tqdm(os.listdir(all_laws_dir)):
#     if file_name.endswith('.txt'):
#         law_name = file_name[:-4]  # '.txt' 확장자 제거
#         file_path = os.path.join(all_laws_dir, file_name)
#         try:
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 content = f.read()
#             all_laws_data.append({'법령명': law_name, '내용': content})
#         except Exception as e:
#             print(f"에러: {file_name} 파일 처리 중 오류 발생 - {e}")

# df_all_laws = pd.DataFrame(all_laws_data)
# print(f"전체 법령 데이터 로드 완료: 총 {len(df_all_laws)}개 법령")


과목별 법령 목록 로드 완료.
'법령_통합텍스트' 폴더에서 전체 법령 텍스트를 로드합니다...


100%|██████████| 1500/1500 [00:00<00:00, 3038.78it/s]

전체 법령 데이터 로드 완료: 총 1500개 법령


In [4]:
# ==============================================================================
# 3. 데이터 전처리 (형태소 분석 및 명사 추출)
# ==============================================================================
print("\n--- 3. 데이터 전처리 시작 (명사 추출) ---")

def preprocess_text(text):
    """Mecab을 이용해 텍스트에서 명사만 추출하고 한 글자 단어는 제거"""
    try:
        # Mecab 사용 시: nouns = tokenizer.nouns(text)
        # Okt 사용 시: nouns = tokenizer.nouns(text)
        nouns = tokenizer.nouns(str(text))
        # 한 글자 명사 제거 후 공백으로 연결
        return ' '.join([noun for noun in nouns if len(noun) > 1])
    except Exception as e:
        # print(f"전처리 오류 발생: {e}, 원문: {text[:30]}")
        return "" # 오류 발생 시 빈 문자열 반환

# '내용' 컬럼에 전처리 함수 적용. 시간이 다소 소요될 수 있습니다.
df_all_laws['내용_전처리'] = df_all_laws['내용'].progress_apply(preprocess_text)
print("데이터 전처리 완료.")
print(df_all_laws[['법령명', '내용_전처리']].head())

df_all_laws['내용_전처리'] = df_all_laws['내용'].progress_apply(preprocess_text)
print("데이터 전처리 완료.")

# --- ★★★ 전처리 결과를 개별 txt 파일로 저장 (이 부분을 추가) ★★★ ---

# 전처리된 파일들을 저장할 새 폴더 이름
preprocessed_output_dir = '법령_전처리_텍스트' 

# 새 폴더 생성
os.makedirs(preprocessed_output_dir, exist_ok=True)
print(f"\n전처리된 텍스트를 '{preprocessed_output_dir}' 폴더에 개별 저장합니다...")

# 데이터프레임을 한 줄씩 순회하며 파일로 저장
for index, row in tqdm(df_all_laws.iterrows(), total=df_all_laws.shape[0]):
    law_name = row['법령명']
    preprocessed_content = row['내용_전처리']
    
    # 출력 파일 경로 설정
    output_path = os.path.join(preprocessed_output_dir, f"{law_name}.txt")
    
    # 전처리된 내용을 .txt 파일로 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(preprocessed_content)

print(f"\n작업 완료! 모든 전처리된 법령이 '{preprocessed_output_dir}' 폴더에 저장되었습니다.")


--- 3. 데이터 전처리 시작 (명사 추출) ---


100%|██████████| 1500/1500 [11:54<00:00,  2.10it/s]


데이터 전처리 완료.
                                        법령명  \
0                10ㆍ27법난 피해자의 명예회복 등에 관한 법률   
1  10ㆍ29이태원참사 피해자 권리보장과 진상규명 및 재발방지를 위한 특별법   
2                     112신고의 운영 및 처리에 관한 법률   
3                           119구조ㆍ구급에 관한 법률   
4                   119긴급신고의 관리 및 운영에 관한 법률   

                                              내용_전처리  
0  목적 법난 관련 피해 사람과 불교 명예 회복 인권 신장 국민 화합 이바지 목적 정의...  
1  총칙 목적 이태원 참사 발생 원인 수습 과정 후속 조치 사실관계 책임 소재 진상 희...  
2  총칙 목적 신고 운영 처리 사항 규정 범죄 각종 사건 사고 상황 국민 생명 신체 재...  
3  총칙 목적 화재 재난 재해 테러 상황 구조 구급 효율 운영 사항 규정 국가 구조 구...  
4  총칙 목적 화재 재난 재해 구조 구급 상황 긴급 신고 관리 운영 사항 신고 개인정보...  


100%|██████████| 1500/1500 [11:48<00:00,  2.12it/s] 


데이터 전처리 완료.

전처리된 텍스트를 '법령_전처리_텍스트' 폴더에 개별 저장합니다...


100%|██████████| 1500/1500 [00:02<00:00, 558.30it/s]


작업 완료! 모든 전처리된 법령이 '법령_전처리_텍스트' 폴더에 저장되었습니다.


In [2]:
import pandas as pd
import os
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. 전처리된 데이터 로드 ---
preprocessed_text_dir = '법령_전처리_텍스트'
all_laws_data = []

print(f"'{preprocessed_text_dir}' 폴더에서 전처리된 법령 텍스트를 로드합니다...")
for filename in tqdm(os.listdir(preprocessed_text_dir)):
    if filename.endswith('.txt'):
        law_name = filename[:-4]
        file_path = os.path.join(preprocessed_text_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        all_laws_data.append({
            '법령명': law_name,
            '내용_전처리': content
        })

df_laws = pd.DataFrame(all_laws_data)
print(f"\n전처리 데이터 로딩 완료: 총 {len(df_laws)}개 법령")


# --- 2. TF-IDF 벡터화 ---
print("\nTF-IDF 벡터화를 진행합니다...")
tfidf_vectorizer = TfidfVectorizer(max_features=10000, min_df=5)
tfidf_vectors = tfidf_vectorizer.fit_transform(df_laws['내용_전처리'])


# --- 3. 유사도 계산 ---
benchmark_laws = ['민법', '형법', '행정기본법']
results_df = df_laws.copy()

for law_name in benchmark_laws:
    try:
        target_idx = df_laws.index[df_laws['법령명'] == law_name].tolist()[0]
        target_vector = tfidf_vectors[target_idx]
        sim_scores = cosine_similarity(tfidf_vectors, target_vector)
        results_df[f'{law_name}_유사도'] = sim_scores
        print(f"'{law_name}'과의 유사도 계산 완료.")

    except IndexError:
        print(f"오류: 기준 법률 '{law_name}'을 찾을 수 없습니다.")


# --- 4. 결과 확인 ---
print("\n\n" + "="*50)
print("                 유사도 분석 결과 확인")
print("="*50)

for law_name in benchmark_laws:
    sim_col = f'{law_name}_유사도'
    if sim_col in results_df.columns:
        top_10 = results_df[results_df['법령명'] != law_name].nlargest(10, sim_col)
        print(f"\n\n--- '{law_name}'과 가장 유사한 법령 TOP 10 ---")
        print(top_10[['법령명', sim_col]].to_string(index=False))


# --- 5. 최종 결과 저장 ---
output_filename = 'tf_idf_전법령_결과_현역.csv'
results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n\n✅ 모든 분석 결과가 '{output_filename}' 파일로 저장되었습니다.")


'법령_전처리_텍스트' 폴더에서 전처리된 법령 텍스트를 로드합니다...


100%|██████████| 5502/5502 [00:00<00:00, 11510.47it/s]



전처리 데이터 로딩 완료: 총 5502개 법령

TF-IDF 벡터화를 진행합니다...
'민법'과의 유사도 계산 완료.
'형법'과의 유사도 계산 완료.
'행정기본법'과의 유사도 계산 완료.


                 유사도 분석 결과 확인


--- '민법'과 가장 유사한 법령 TOP 10 ---
               법령명   민법_유사도
            가사소송규칙 0.517947
             가사소송법 0.514872
             민사집행법 0.353149
       후견등기에 관한 법률 0.337079
              국제사법 0.320565
            민사집행규칙 0.318835
채무자 회생 및 파산에 관한 법률 0.316726
                상법 0.308696
       후견등기에 관한 규칙 0.298736
 가족관계의 등록 등에 관한 법률 0.281660


--- '형법'과 가장 유사한 법령 TOP 10 ---
                             법령명   형법_유사도
                             군형법 0.635891
              특정범죄 가중처벌 등에 관한 법률 0.604259
                           국가보안법 0.527538
                폭력행위 등 처벌에 관한 법률 0.473335
            특정경제범죄 가중처벌 등에 관한 법률 0.374519
                           밀항단속법 0.368587
               보건범죄 단속에 관한 특별조치법 0.336398
             성폭력범죄의 처벌 등에 관한 특례법 0.328000
선박 및 해상구조물에 대한 위해행위의 처벌 등에 관한 법률 0.306891
                         조세범 처벌법 0.301599


--- '행정기본법'과 가장 유사한 법령 T

In [3]:
import pandas as pd
import os
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# --- 1. 원본 텍스트 데이터 로드 ---
# SBERT는 원본 텍스트를 사용하므로, 원본 .txt 파일이 있는 폴더를 지정합니다.
original_text_dir = '법령_통합텍스트(현역)'
all_laws_data = []

print(f"'{original_text_dir}' 폴더에서 원본 법령 텍스트를 로드합니다...")
if not os.path.isdir(original_text_dir):
    print(f"오류: '{original_text_dir}' 폴더를 찾을 수 없습니다. 원본 txt 통합 파일이 준비되어 있는지 확인해주세요.")
    exit()

for filename in tqdm(os.listdir(original_text_dir)):
    if filename.endswith('.txt'):
        law_name = filename[:-4]
        file_path = os.path.join(original_text_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        all_laws_data.append({
            '법령명': law_name,
            '내용': content  # SBERT는 전처리 전 '내용' 컬럼을 사용
        })

df_laws = pd.DataFrame(all_laws_data)
print(f"\n원본 데이터 로딩 완료: 총 {len(df_laws)}개 법령")


# --- 2. SBERT 벡터화 ---
print("\nSBERT 벡터화를 진행합니다. (시간이 많이 소요될 수 있습니다 ⏳)")
# GPU 자동 감지
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"SBERT 분석에 사용할 장치: {device.upper()}")

# SBERT 모델 로드
sbert_model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    device=device
)
# 원본 '내용' 컬럼을 인코딩
sbert_vectors = sbert_model.encode(
    df_laws['내용'].tolist(),
    show_progress_bar=True,
    batch_size=32
)


# --- 3. 유사도 계산 ---
benchmark_laws = ['민법', '형법', '행정기본법']
results_df = df_laws.copy()

for law_name in benchmark_laws:
    try:
        # 기준 법률의 인덱스 찾기
        target_idx = df_laws.index[df_laws['법령명'] == law_name].tolist()[0]
        # 해당 인덱스의 SBERT 벡터 추출
        target_vector = sbert_vectors[target_idx]

        # 전체 SBERT 벡터와 기준 벡터 간의 코사인 유사도 계산
        sim_scores = cosine_similarity(sbert_vectors, target_vector.reshape(1, -1))

        # 결과를 데이터프레임의 새 컬럼으로 추가
        results_df[f'{law_name}_유사도_SBERT'] = sim_scores
        print(f"'{law_name}'과의 유사도 계산 완료.")

    except IndexError:
        print(f"오류: 기준 법률 '{law_name}'을 찾을 수 없습니다.")
    except Exception as e:
        print(f"'{law_name}' 처리 중 오류 발생: {e}")


# --- 4. 결과 확인 ---
print("\n\n" + "="*50)
print("              SBERT 기반 유사도 분석 결과 확인")
print("="*50)

for law_name in benchmark_laws:
    sim_col = f'{law_name}_유사도_SBERT'
    if sim_col in results_df.columns:
        # 자기 자신을 제외하고 상위 10개 추출
        top_10 = results_df[results_df['법령명'] != law_name].nlargest(10, sim_col)
        print(f"\n\n--- '{law_name}'과 가장 유사한 법령 TOP 10 (SBERT) ---")
        print(top_10[['법령명', sim_col]].to_string(index=False))


# --- 5. 최종 결과 저장 ---
output_filename = 'sbert_전법령_결과_현역.csv'
# 원본 '내용' 컬럼은 용량이 크므로 제외하고 저장
results_df.drop(columns=['내용']).to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n\n✅ 모든 SBERT 분석 결과가 '{output_filename}' 파일로 저장되었습니다.")

c:\Users\silve\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'법령_통합텍스트(현역)' 폴더에서 원본 법령 텍스트를 로드합니다...


100%|██████████| 5502/5502 [00:00<00:00, 7253.61it/s]



원본 데이터 로딩 완료: 총 5502개 법령

SBERT 벡터화를 진행합니다. (시간이 많이 소요될 수 있습니다 ⏳)
SBERT 분석에 사용할 장치: CUDA


Batches: 100%|██████████| 172/172 [00:22<00:00,  7.64it/s]


'민법'과의 유사도 계산 완료.
'형법'과의 유사도 계산 완료.
'행정기본법'과의 유사도 계산 완료.


              SBERT 기반 유사도 분석 결과 확인


--- '민법'과 가장 유사한 법령 TOP 10 (SBERT) ---
                    법령명  민법_유사도_SBERT
                  법원조직법      0.834385
                 행정소송규칙      0.816781
                 민사소송규칙      0.816720
대법원 공직자윤리위원회의 권한에 관한 규칙      0.815048
                 가사소송규칙      0.814355
                  형사소송법      0.804145
                     상법      0.802892
            민사소송 등 인지규칙      0.802111
           법원행정처 행정심판규칙      0.801739
                 민사집행규칙      0.801647


--- '형법'과 가장 유사한 법령 TOP 10 (SBERT) ---
                         법령명  형법_유사도_SBERT
                 경범죄 처벌법 시행령      0.868211
      범죄인인도법에의한인도심사등의절차에관한규칙      0.842682
                         사면법      0.839862
                경범죄 처벌법 시행규칙      0.833647
  스토킹범죄 사건의 심리 및 재판 등에 관한 규칙      0.829639
범죄수익은닉의 규제 및 처벌 등에 관한 법률 시행령      0.826938
                     경범죄 처벌법      0.817388
                       법관징계법      0.817092
                  